In [1]:
import numpy as np
import sklearn
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from matplotlib import rc
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from matplotlib.ticker import MaxNLocator
import scipy.signal
from xgboost.sklearn import XGBClassifier
import pickle
import os

/mnt/raid-cita/nhussain/venv-2.7.13/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
xgbmodel = XGBClassifier(seed = 0, n_estimators=225, max_depth=9, subsample=0.9)

In [3]:
df = pd.read_csv('../csvs/short_integration_features.csv', index_col=0)
del df['instability_time']
del df['Rel_Eerr_short']
#Nrows = df.shape[0]
#print Nrows, len(np.where(df['Stable']==0)[0])
print df.shape
df.head()

(14999, 116)


,runstring,Stable,Rel_Eerr,t_final_short,avg_a1,std_a1,max_a1,min_a1,norm_std_a1,norm_max_a1,...,avg_iH3,std_iH3,max_iH3,min_iH3,avg_ecross3,std_ecross3,max_ecross3,min_ecross3,norm_a3_slope,Lyapunov_time
0,0000000.bin,0.0,1.099985e-04,10000.0,1.000165,0.000329,1.001248,0.999691,0.000329,0.001083,...,4.706365,0.263380,5.264568,4.410404,0.297364,0.016641,0.339655,0.010871,2.681898e-11,9041.175397
1,0000001.bin,0.0,1.388587e-08,10000.0,0.999952,0.000739,1.002089,0.998309,0.000739,0.002137,...,1.777638,0.610370,2.507803,0.588028,0.115674,0.039718,0.251880,0.137917,-9.314702e-11,2192.382019
2,0000002.bin,0.0,3.048261e-04,10000.0,0.998498,0.001948,1.002043,0.995446,0.001951,0.003550,...,0.982187,0.060058,1.060480,0.875622,0.038930,0.002380,0.026117,0.007227,-4.095504e-11,2191.525922
3,0000003.bin,1.0,6.492574e-10,10000.0,1.000006,0.000026,1.000126,0.999973,0.000026,0.000120,...,9.848375,0.070031,9.914136,9.650551,0.514386,0.003658,0.301072,0.279903,-7.257099e-12,58599.215966
4,0000004.bin,0.0,9.336734e-07,10000.0,1.067267,0.030186,1.121364,1.000000,0.028284,0.063028,...,1.453843,0.726529,2.660701,0.154157,0.496372,0.248052,1.358604,0.261358,-2.610208e-08,43524.152375


In [4]:

features = []
features += ['avg_iH1', 'avg_iH2']
features += ['norm_std_a1', 'norm_max_a1', 'norm_std_window10_a1', 'norm_max_window10_a1']
features += ['norm_std_a2', 'norm_max_a2', 'norm_std_window10_a2', 'norm_max_window10_a2']
features += ['norm_std_a3', 'norm_max_a3', 'norm_std_window10_a3', 'norm_max_window10_a3']

features += ['avg_ecross1', 'std_ecross1', 'max_ecross1', 'min_ecross1']
features += ['avg_ecross2', 'std_ecross2', 'max_ecross2', 'min_ecross2']
features += ['avg_ecross3', 'std_ecross3', 'max_ecross3', 'min_ecross3']

features += ['norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope']
features += ['Lyapunov_time']

y = df['Stable']
X = df[features]
X.head()

,avg_iH1,avg_iH2,norm_std_a1,norm_max_a1,norm_std_window10_a1,norm_max_window10_a1,norm_std_a2,norm_max_a2,norm_std_window10_a2,norm_max_window10_a2,...,max_ecross2,min_ecross2,avg_ecross3,std_ecross3,max_ecross3,min_ecross3,norm_a1_slope,norm_a2_slope,norm_a3_slope,Lyapunov_time
0,0.396282,0.481704,0.000329,0.001083,0.000214,0.000456,0.000701,0.001978,0.000446,0.000964,...,0.333921,0.077775,0.297364,0.016641,0.339655,0.010871,-9.963524e-11,2.144832e-10,2.681898e-11,9041.175397
1,1.034575,0.823935,0.000739,0.002137,0.000202,0.000463,0.000200,0.000350,0.000226,0.000377,...,0.307149,0.075092,0.115674,0.039718,0.251880,0.137917,-4.938423e-11,3.559377e-11,-9.314702e-11,2192.382019
2,0.440470,0.491261,0.001951,0.003550,0.001735,0.002860,0.002303,0.004034,0.002080,0.003409,...,0.583040,0.328325,0.038930,0.002380,0.026117,0.007227,6.451005e-11,-3.330088e-11,-4.095504e-11,2191.525922
3,0.432098,0.550624,0.000026,0.000120,0.000005,0.000009,0.000019,0.000050,0.000016,0.000041,...,0.048402,0.000124,0.514386,0.003658,0.301072,0.279903,8.676076e-12,7.816527e-13,-7.257099e-12,58599.215966
4,5.722053,17.442426,0.028284,0.063028,0.004313,0.010716,0.038832,0.160685,0.090892,0.112168,...,9.321252,0.823132,0.496372,0.248052,1.358604,0.261358,6.816926e-08,3.913858e-08,-2.610208e-08,43524.152375


In [5]:
Nrows = int(0.4*X.shape[0])
trainX = X.iloc[:Nrows, :]
trainY = y.iloc[:Nrows]
testX = X.iloc[Nrows:, :]
testY = y.iloc[Nrows:]

In [ ]:
xgbmodel.fit(trainX, trainY)

In [8]:
pickle.dump(xgbmodel ,open("OptimalXGBmodel_SepFile.pkl", 'wb'))
statinfo = os.stat('OptimalXGBmodel_SepFile.pkl')
print statinfo.st_size

832964


In [9]:
loaded_model = pickle.load(open("OptimalXGBmodel_SepFile.pkl", 'rb'))

In [10]:
#new_feat= features[:] #not make an alias, but make a new copy

new_feat= []
fnew_feat += ['avg_iH1', 'avg_iH2']
features += ['norm_std_a1', 'norm_max_a1', 'norm_std_window10_a1', 'norm_max_window10_a1']
features += ['norm_std_a2', 'norm_max_a2', 'norm_std_window10_a2', 'norm_max_window10_a2']
features += ['norm_std_a3', 'norm_max_a3', 'norm_std_window10_a3', 'norm_max_window10_a3']

features += ['avg_ecross1', 'std_ecross1', 'max_ecross1', 'min_ecross1']
features += ['avg_ecross2', 'std_ecross2', 'max_ecross2', 'min_ecross2']
features += ['avg_ecross3', 'std_ecross3', 'max_ecross3', 'min_ecross3']

features += ['norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope']
features += ['Lyapunov_time']

y_new = df['Stable']
X_new = df[new_feat]
X_new.head()

,avg_iH1,avg_iH2,norm_std_a1,norm_max_a1,norm_std_window10_a1,norm_max_window10_a1,norm_std_a2,norm_max_a2,norm_std_window10_a2,norm_max_window10_a2,...,max_ecross2,min_ecross2,avg_ecross3,std_ecross3,max_ecross3,min_ecross3,norm_a1_slope,norm_a2_slope,norm_a3_slope,Lyapunov_time
0,0.396282,0.481704,0.000329,0.001083,0.000214,0.000456,0.000701,0.001978,0.000446,0.000964,...,0.333921,0.077775,0.297364,0.016641,0.339655,0.010871,-9.963524e-11,2.144832e-10,2.681898e-11,9041.175397
1,1.034575,0.823935,0.000739,0.002137,0.000202,0.000463,0.000200,0.000350,0.000226,0.000377,...,0.307149,0.075092,0.115674,0.039718,0.251880,0.137917,-4.938423e-11,3.559377e-11,-9.314702e-11,2192.382019
2,0.440470,0.491261,0.001951,0.003550,0.001735,0.002860,0.002303,0.004034,0.002080,0.003409,...,0.583040,0.328325,0.038930,0.002380,0.026117,0.007227,6.451005e-11,-3.330088e-11,-4.095504e-11,2191.525922
3,0.432098,0.550624,0.000026,0.000120,0.000005,0.000009,0.000019,0.000050,0.000016,0.000041,...,0.048402,0.000124,0.514386,0.003658,0.301072,0.279903,8.676076e-12,7.816527e-13,-7.257099e-12,58599.215966
4,5.722053,17.442426,0.028284,0.063028,0.004313,0.010716,0.038832,0.160685,0.090892,0.112168,...,9.321252,0.823132,0.496372,0.248052,1.358604,0.261358,6.816926e-08,3.913858e-08,-2.610208e-08,43524.152375


In [11]:
#generate the same train and test split
Nrows = int(0.4*X_new.shape[0])
trainX = X_new.iloc[:Nrows, :]
trainY = y_new.iloc[:Nrows]
testX = X_new.iloc[Nrows:, :]
testY = y_new.iloc[Nrows:]

In [19]:
print testX.shape
loaded_model.predict(testX)
#loaded_model.predict(testX.iloc[:1,:])

(9000, 30)


array([ 0.,  0.,  0., ...,  0.,  1.,  0.])